In [1]:
import json
import os
import re
from typing import List, Dict

import pandas as pd

In [2]:
# 사용할 엔티티 타입
ENTITY_TYPES = ["PS", "LC", "OG", "DT", "TI", "QT"]

# <|PS|>엔티티<|PS|> 같은 패턴 찾는 정규식
ENTITY_PATTERN = re.compile(
    r"<\|(PS|LC|OG|DT|TI|QT)\|>(.+?)<\|\1\|>",
    flags=re.DOTALL
)

In [3]:
def parse_tagged_sentence(tagged_text: str) -> Dict:
    """
    라벨링된 문장 하나를 받아서
    - 태그가 제거된 실제 문장 text
    - 엔티티 스팬 리스트 [{"start": int, "end": int, "label": "PS"}, ...]
    를 반환합니다.
    """
    entities = []
    clean_text_parts = []
    curr_pos = 0  # clean_text 상의 현재 길이

    last_end = 0
    for match in ENTITY_PATTERN.finditer(tagged_text):
        label = match.group(1)
        surface = match.group(2)

        # 태그 이전의 일반 텍스트 추가
        before = tagged_text[last_end:match.start()]
        if before:
            clean_text_parts.append(before)
            curr_pos += len(before)

        # 엔티티 텍스트 추가
        ent_start = curr_pos
        clean_text_parts.append(surface)
        curr_pos += len(surface)
        ent_end = curr_pos

        entities.append({
            "start": ent_start,
            "end": ent_end,
            "label": label
        })

        last_end = match.end()

    # 마지막 태그 뒤에 남은 텍스트
    tail = tagged_text[last_end:]
    if tail:
        clean_text_parts.append(tail)
        curr_pos += len(tail)

    clean_text = "".join(clean_text_parts)

    return {
        "text": clean_text,
        "entities": entities
    }

In [6]:
# ---- 예시 테스트 ----
sample = """“반갑다. 힘찬 호랑이 해야” <|DT|>2022년<|DT|> 임인년(壬寅年) 첫날이 밝았다."""
parsed = parse_tagged_sentence(sample)
print(parsed["text"])
print(parsed["entities"])

for ent in parsed['entities']:
    print(parsed['text'][ent['start']:ent['end']])

“반갑다. 힘찬 호랑이 해야” 2022년 임인년(壬寅年) 첫날이 밝았다.
[{'start': 17, 'end': 22, 'label': 'DT'}]
2022년


# 1. Process data

In [16]:
dataset_name = "NIKL_NEWSPAPER_2023_CSV"
fname = "NEWSPAPER_2022_1"
sample_name = "sample1"

save_base_path = f"results/{dataset_name}-{fname}-{sample_name}"

result_df = pd.read_parquet(f"{save_base_path}/entities.parquet")

# 2. Save to disk as datasets

In [9]:
from datasets import Dataset

In [10]:
raw_dataset = Dataset.from_list([parsed, parsed])

In [11]:
raw_dataset

Dataset({
    features: ['text', 'entities'],
    num_rows: 2
})